In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sepal_ui import sepalwidgets as sw
from sepal_ui import aoi
from sepal_ui.mapping import SepalMap
from sepal_ui.scripts import utils as su
from sepal_ui.mapping.inspector_control import InspectorControl
from sepal_ui.mapping.draw_control import DrawControl

import ipyvuetify as v
import ee
su.init_ee()

from datetime import datetime, date
from math import floor, ceil

<IPython.core.display.Javascript object>

ResizeTrigger()

In [6]:
from sepal_ui import aoi
aoi_view = aoi.AoiView(
            gee=True,
            map_=map_1,
            methods=["ADMIN1", "ADMIN2", "SHAPE", "DRAW", "ASSET"],
        )
aoi_view

AoiView(children=[MethodSelect(dense=True, items=[{'header': 'Administrative definitions'}, {'text': 'Admin le…

In [8]:
aoi_view.model

AoiModel(admin=2327, asset_name=None, method=ADMIN1, name=PRY_San_Pedro, asset_json=None, vector_json=None, point_json=None, geo_json={'type': 'FeatureCollection', 'features': []})

In [3]:
map_1 = SepalMap(vinspector= True)
map_1.add_basemap("SATELLITE")
dc = DrawControl(map_1)
map_1.add(dc)
map_1

SepalMap(center=[0, 0], controls=(ZoomControl(options=['position', 'transparent_bg'], position='topright', wid…

In [ ]:
aoi = ee.FeatureCollection(dc.to_json())
start = '2024-12-01'
end = '2025-01-02'

In [ ]:
def _from_radd(start, end, aoi):
    """reformat the radd alerts to fit the module expectation"""

    # extract dates from parameters
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")

    # select the alerts and mosaic them as image
    source = "projects/radar-wur/raddalert/v1"
    alerts = (
        ee.ImageCollection(source)
        .filterBounds(aoi)
        .filterMetadata("layer", "contains", "alert")
        .mosaic()
        .uint16()
    )

    # filter the alerts dates
    # extract julian dates ()
    start = int(start.strftime("%y%j"))
    end = int(end.strftime("%y%j"))

    # masked all the images that are not between the limits dates
    alerts = alerts.updateMask(
        alerts.select("Date").gt(start).And(alerts.select("Date").lt(end))
    )

    # create a unique alert band
    alert_band = (
        alerts.select("Alert").remap([0, 1, 2, 3], [0, 0, 2, 1]).rename("alert")
    )

    # change the date format
    date_band = alerts.select("Date").divide(1000).add(2000).rename("date")

    # create the composit image
    all_alerts = alerts.addBands(alert_band).addBands(date_band)

    return all_alerts.clip(aoi)

In [ ]:
test = _from_radd(start, end, aoi)

In [ ]:
map_2 = SepalMap(vinspector= True)
map_2.add_basemap("SATELLITE")
map_2.addLayer(ee_object=aoi, name="aoi")
map_2.addLayer(test, name="rad")
map_2

In [ ]:
#Inputs
clip_alert = test
# Generar 3 bandas para las alertas de todo el area
i = clip_alert.select("alert").gt(0).rename("mask_alert")
a = clip_alert.select("alert")
d = clip_alert.select("date")
# alertarea = i.multiply(ee.Image.pixelArea()).rename('area');

alerta_reducir = i.addBands(i).addBands(a).addBands(d)
map_2.addLayer(alerta_reducir, name="alerta_reducir")


In [ ]:

custom_reducer = (
    ee.Reducer.count()
    .combine(ee.Reducer.mean().unweighted(), "alert_type_")
    .combine(ee.Reducer.minMax().unweighted(), "alert_date_")
)


In [ ]:
alert_raster = test

   #reducer, geometry, scale, geometryType, eightConnected, labelProperty, crs, crsTransform, bestEffort, maxPixels, tileScale, geometryInNativeProjection
bb = alerta_reducir.reduceToVectors(
    reducer=custom_reducer,
    geometry=aoi,
    scale=15,
    geometryType="bb",
    #eightConnected=True,
    #crs= 'EPSG:4326',
    maxPixels=1e13,
    tileScale=1,
).getInfo()

print(bb)